In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
def get_raw_data(file_name):
    data = loadmat(file_name)
    train = data['X']
    test = data['T']
    return data, train, test

def process_raw_data(x):
    x_ = np.copy(x).transpose((2, 1, 0))
    return x_

In [3]:
def calc_design_matrix(x, c, h):
    '''
    params x: array_like, shape is (n_features, n_sapmles)
    '''
    print(x.shape)
    print(c.shape)
    print(x[:, None, :].shape)
    print(c[None].shape)
    print((x[:, None, :] - c[None]).shape)
    return np.exp(((x[:, None, :] - c[None])**2).sum(axis=2) / (2 * h**2))

In [4]:
def train_model(x, h):
    n_category = x.shape[0]
    Theta = np.zeros((n_category, n_category, x.shape[2]))
    y = np.concatenate([np.ones(x.shape[1]), -np.ones(x.shape[1])])
    for i in range(n_category):
        for j in range(n_category):
            if i == j:
                continue
            tmp_x = np.concatenate([x[i, :, :], x[j, :, :]], axis=0)
            Phi = calc_design_matrix(tmp_x, tmp_x, h)
            Theta[i][j] = np.linalg.solve(Phi.T.dot(Phi), Phi.t.dot(y))
    return Theta



# def train_model(x, h):
#     '''
#     params x: array_like, shape is (n_features, n_samples, n_categories)
#     params h: float, hyperparameter(variance of Gaussian std error)
#     '''
#     tmp_x = np.copy(x).reshape(x.shape[0] * x.shape[1], -1)
#     print("ok")
#     Phi = calc_design_matrix(tmp_x, tmp_x, h)
#     print("Phi.shape: ", Phi.shape)
#     PhiT_Phi = Phi.T.dot(Phi)
#     Theta = np.zeros((x.shape[0], x.shape[2]))
#     category = np.arange(0, tmp_x.shape[0]).astype("int32") // x.shape[1]
#     for i in range(x.shape[0]):
#         y = np.where(category == i, 1.0, -1.0)
#         Theta[i] = np.linalg.solve(PhiT_Phi, Phi.T.dot(y))
    
#     return Theta

def pred(Theta, x):
    '''
    params Theta: parameters of a model learnd, (n_category, n_category, n_feature)
    params x: data, (n_samples, n_features)
    '''
    p = np.zeros((x.shape[0], x.shape[1]))
    for c in range(x.shape[0]):
        x_c = x[c, :, :]
        phi = calc_design_matrix(x_c, )
        f = np.dot(Theta, x_c.T)
        p[c,:] = np.where(f > 0, 1, 0).sum(axis=1).argmax(axis=0)
    return p



In [5]:
data_raw, train_raw, test_raw = get_raw_data("ADA4-digit.mat")

train = process_raw_data(train_raw)
test = process_raw_data(test_raw)
print(train.shape)
print(test.shape)

(10, 500, 256)
(10, 200, 256)


In [6]:
Theta = train_model(train, h=0.1)

TypeError: calc_design_matrix() missing 1 required positional argument: 'h'